In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.storage import LocalFileStore
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

import time

llm = ChatOpenAI(
    # model_name="gpt-3.5-turbo"
    # , 
    temperature=0.1
    , streaming=True
)

memory = ConversationBufferMemory(
    memory_key = "chat_history"
    , return_messages = True
    , llm = llm
)

# 문서로드
loader = UnstructuredFileLoader("./files/document.txt")

# 쪼개기
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

docs = loader.load_and_split(text_splitter=splitter)

# 임베딩 / 캐시
embeddings = OpenAIEmbeddings()

cache_dir = LocalFileStore("./.cache/")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# 벡터
vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\slive\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\slive\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [2]:
def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]

    results = []
    for doc in documents:
        result = map_doc_chain.invoke({"context": doc.page_content, "question": question}).content
        results.append(result)
        time.sleep(20)

    return "\n\n".join(results)


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

In [3]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = {
    "context": map_chain,
    "question": RunnablePassthrough(),
    "history": load_memory
} | final_prompt | llm

In [4]:
def invoke_chain(question):
    result = chain.invoke(question).content
    memory.save_context(
        {"input": question},
        {"output": result},
    )
    print(result)

In [5]:
invoke_chain("Is Aaronson guilty?")

Yes, Aaronson was guilty of the crimes he was charged with.


In [6]:
invoke_chain("What message did he write in the table?")

He wrote: 2+2=5


In [7]:
invoke_chain("Who is Julia?")

Julia is a character mentioned in the text provided. She is someone who was with the main character in the Ministry of Love and was part of his attempt to resist the power of the Party.
